<a href="https://colab.research.google.com/github/Wittgenbot/fine-tuning/blob/main/finetuned_model_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

## Package Installs

In [ ]:
! pip install -q -U transformers
! pip install -q -U accelerate
! pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## Imports

In [ ]:
import torch
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Model

## Configuration

In [ ]:
model_name = 'descartesevildemon/witt'

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True)

## Load Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map='auto',
                                             trust_remote_code=True)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Load and Configure Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = False
tokenizer.add_bos_token = False

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

# Model Manual Tests

## Response Generation Function

In [ ]:
def get_completion(question):

  prompt_template = """
  <s>
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about Ludwig Wittgenstein in a conversational, clear and coherent tone:
  {question}
  [/INST]
  """
  prompt = prompt_template.format(question=question)

  encodeds = tokenizer(prompt,
                       return_tensors="pt",
                       add_special_tokens=True)

  model_inputs = encodeds.to('cuda:0')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=1000,
                                 do_sample=True,
                                 top_p=0.9,
                                 temperature=0.7,
                                 pad_token_id=tokenizer.eos_token_id)

  decoded = tokenizer.batch_decode(generated_ids)

  response = decoded[0]

  formatted_response = ''
  lines = response.split('\n')
  for line in lines:
      if len(line) > 125:
          w = textwrap.TextWrapper(width=125, break_long_words=False)
          line = '\n'.join(w.wrap(line))
      formatted_response += line + '\n'

  print(formatted_response)

## Manual Tests

In [ ]:
prompt = 'According to Ludwig Wittgenstein\'s philosophy, is the existence of a private language possible?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  According to Ludwig Wittgenstein's philosophy, is the existence of a private language possible?
  [/INST]
  
  Wittgenstein's philosophy, as expressed in his later work, particularly in the Philosophical Investigations, argues against
the possibility of a private language. A private language, in this context, is a language that only one person uses and
understands, with meanings that are not shared or publicly accessible.

Wittgenstein's argument against a private language is based on the idea of language as a social phenomenon. He believes that
the meaning of a word is determined by its use in a linguistic community, and that for a word to have meaning, it must be
part of a shared system of language that can be understood and interpreted by others.

In a private language, there woul

In [ ]:
prompt = 'What determines the meaning of a word?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What determines the meaning of a word?
  [/INST]
  
  The meaning of a word, according to Ludwig Wittgenstein, is not determined by some inherent property or a private mental
image, but rather by its use in the context of a language game. Wittgenstein introduced the concept of language games to
emphasize that language is a tool used in various social contexts, and the meaning of a word is derived from the rules of the
particular language game in which it is used.

For instance, the word "game" itself has different meanings depending on the context. In the language game of sports, "game"
refers to a competitive activity with rules. In the language game of entertainment, "game" could refer to a form of amusement
or recreation. The meaning of the word "game" is determined by the specifi

In [ ]:
prompt = 'According to Ludwig Wittgenstein, when can something can be classified as a game?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  According to Ludwig Wittgenstein, when can something can be classified as a game?
  [/INST]
  
  Wittgenstein's perspective on what can be classified as a game is outlined in his later philosophical work, particularly in
the "Philosophical Investigations." He argues against the idea of a single, definitive essence that all games share, instead
emphasizing the variety of activities we call games and the importance of language games in understanding this concept.

Wittgenstein uses the example of games to illustrate his theory of language games, which is the idea that the meaning of a
word is determined by its use in a specific language game—a social activity governed by a set of rules. According to him, the
concept of a game is not tied to any particular set of characteristics but rat

In [ ]:
prompt = 'How do philosophical problems arise from misunderstandings of language?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  How do philosophical problems arise from misunderstandings of language?
  [/INST]
  
  Philosophical problems often arise from misunderstandings of language due to the fact that language is a tool we use to
represent and communicate about the world, but it doesn't directly correspond to the world itself. Wittgenstein, in his later
philosophy, emphasized this point through the idea of language games. He argued that words and phrases get their meaning from
the rules of the specific language game in which they are used, and these rules can vary greatly from one context to another.

For instance, consider the word "game". In the language game of sports, a game is a competitive activity with rules. But in
the language game of board games, a game is a recreational activity played on a boar

In [ ]:
prompt = 'Can you provide examples of how philosophical problems arise from the misuse of language?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Can you provide examples of how philosophical problems arise from the misuse of language?
  [/INST]
  
  Absolutely, Ludwig Wittgenstein, in his later philosophy, particularly in the "Philosophical Investigations," argued that
many philosophical problems arise from the misuse of language. He introduced the idea of language games, which are specific
activities where language is used in a particular way. According to him, when we step out of the bounds of these language
games, we encounter confusion and philosophical problems.

For instance, consider the problem of other minds. We can never directly experience another person's thoughts or feelings.
When we say we know what others are thinking or feeling, we're using language in a way that goes beyond the language game of
personal exper

In [ ]:
prompt = 'In what ways does the examination of pain\'s sameness challenge the understanding of subjective experiences?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  In what ways does the examination of pain's sameness challenge the understanding of subjective experiences?
  [/INST]
  
  The examination of pain's sameness, as discussed by Ludwig Wittgenstein in his philosophical work, particularly in the
Philosophical Investigations, challenges the understanding of subjective experiences in several ways.

Firstly, Wittgenstein questions the idea of a private language, a language that only one person understands, which is often
used to describe subjective experiences like pain. He argues that for a word to have meaning, it must be part of a public
language, a language that is shared and understood by a community. However, pain is a deeply personal experience, making it
difficult to apply the rules of a public language to it.

Secondly, Wittgenstei

In [ ]:
prompt = 'What is the purpose of the analogy with the toolbox?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is the purpose of the analogy with the toolbox?
  [/INST]
  
  The analogy with the toolbox is a concept that Ludwig Wittgenstein uses in his later philosophy, particularly in his work
"Philosophical Investigations." The purpose of this analogy is to illustrate his idea of language as a toolbox, where words
and phrases function like tools in a toolbox, each having a specific use and purpose.

Wittgenstein argues that the meaning of a word is not its reference to an object in the world, but its use in a language-
game, a concept he also introduces in "Philosophical Investigations." Language-games are specific contexts or situations
where language is used. For example, the word "game" is used differently in the language-game of describing board games
compared to the language-game 

In [ ]:
prompt = 'What does following a rule entail?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does following a rule entail?
  [/INST]
  
  Following a rule, according to Ludwig Wittgenstein, is a complex and context-dependent activity. It's not just a matter of
applying a formula or algorithm in a mechanical way. Instead, it involves understanding the rule in a particular way, and
then acting on that understanding in a manner that's consistent with it.

Wittgenstein, in his later philosophy, emphasized the importance of language games and forms of life in understanding rule-
following. He argued that rules don't have a life of their own; they're embedded in the practices and activities of a
community. So, when we learn to follow a rule, we're not just internalizing a set of instructions; we're learning to
participate in a social practice.

For Wittgenstein, the meaning o

In [ ]:
prompt = 'Can you describe Wittgenstein\'s picture theory of meaning?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Can you describe Wittgenstein's picture theory of meaning?
  [/INST]
  
  Absolutely, I'd be happy to explain Wittgenstein's picture theory of meaning. This theory is a fundamental concept in his
early philosophy, particularly as presented in the Tractatus Logico-Philosophicus.

The picture theory of meaning suggests that language functions like a picture of the world. Just as a picture is a model of
reality, language is a model of the facts that make up the world. According to Wittgenstein, propositions (i.e., sentences)
in language can depict the world accurately if they have the same logical structure as the states of affairs in the world.

In simpler terms, Wittgenstein believed that words in a sentence correspond to objects or facts in the world. When we make a
statement, we're 

In [ ]:
prompt = 'What does Wittgenstein say about tautologies and contradictions in the Tractatus?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does Wittgenstein say about tautologies and contradictions in the Tractatus?
  [/INST]
  
  Wittgenstein, in the Tractatus Logico-Philosophicus, discusses tautologies and contradictions as part of his exploration of
the nature of logic and language. Tautologies and contradictions are significant in Wittgenstein's work because they
represent the limits of meaningful language and thought.

A tautology, according to Wittgenstein, is a statement that is true in all possible circumstances. For example, the statement
"Either it is raining or it is not raining" is a tautology. It is true regardless of the actual state of the world.
Wittgenstein argues that tautologies do not convey any substantial information about the world, as their truth is a mere
consequence of their logical struct

In [ ]:
prompt = 'Why is the notion of a criterion for correctness important for Wittgenstein\'s private language argument?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  Why is the notion of a criterion for correctness important for Wittgenstein's private language argument?
  [/INST]
  
  The notion of a criterion for correctness is crucial to understanding Wittgenstein's private language argument, which he
presented in his later philosophical work, particularly in the Philosophical Investigations.

Wittgenstein's private language argument challenges the idea that individuals can have a private language, a language that
only they understand and use. He argues that for language to be meaningful, it must be public and shareable, meaning that the
words and concepts used must be accessible and understandable by others.

The concept of a criterion for correctness is central to this argument. Wittgenstein argues that for a word or concept to
have meaning, 

In [ ]:
prompt = 'What does Wittgenstein say about aspect perception?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What does Wittgenstein say about aspect perception?
  [/INST]
  
  Wittgenstein, in his later philosophy, particularly in the "Philosophical Investigations," explores the concept of aspect
perception, which is the ability to see something as something else, or to experience a shift in the way we perceive or
interpret a particular image or situation.

He uses the famous example of seeing a drawing of a cube as either a three-dimensional object or a two-dimensional picture,
depending on how we're prompted to see it. This example illustrates that our perception is not solely determined by the
object itself, but also by our mental frameworks and the concepts we apply.

Wittgenstein argues that aspect perception is not a matter of objective truth, but rather a subjective experience. He cl

In [ ]:
prompt = 'What is the paradox of ostension?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is the paradox of ostension?
  [/INST]
  
  The paradox of ostension is a concept that arises in Ludwig Wittgenstein's later philosophy, particularly in his work
"Philosophical Investigations." It's a paradox related to the way words connect with the world and how we use language to
represent reality.

The term "ostension" refers to the act of pointing or showing something, with the intention of conveying its meaning or
identity. For instance, if I point at a apple and say "this is an apple," I'm using ostension to teach you what an apple is.

The paradox arises when we consider the limitations of ostension. If I were to point at a word, like "apple," and say "this
is what we call 'apple' in English," I'm using ostension to show the word, but I'm not actually showing the concept

In [ ]:
prompt = 'What is Wittgenstein trying to show with the builders\' language example?'
get_completion(prompt)


  <s> 
  [INST]
  You are a philosophy professor specialized in Ludwig Wittgenstein. Your task is to answer the following question about
Ludwig Wittgenstein in a conversational, clear and coherent tone:
  What is Wittgenstein trying to show with the builders' language example?
  [/INST]
  
  Wittgenstein's builders' language example is a thought experiment introduced in his later philosophical work, particularly
in the Philosophical Investigations. The example is used to illustrate his concept of language games and the idea that
meaning is derived from use and context, rather than from some inherent property of words themselves.

In the builders' language example, Wittgenstein imagines a scenario where two workers are building a house. One worker, the
builder, is only allowed to use a few basic commands like "slab," "pillar," "block," and "beam" to instruct the other worker,
the apprentice, on what to do. The apprentice can only respond with yes or no, acknowledging or rejecting the c